# **PLOTLY GRÁFICAS**

# *Importacion y utilizacion de las librerias para compilar el codigo*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica
import plotly.express as px

# *Data import*

In [2]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
# Imprimir la ruta relativa
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

### *Creación de los df por cada archivo json*

In [3]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres para obtener solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Imprimir la lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_puebla', 'df_alfa_q_sep_2023_pachuca', 'df_alfa_q_tulancingo', 'df_enero_2024_querertaro', 'df_feb_2024_puebla', 'df_feb_2024_queretaro', 'df_financiamientos_2019_puebla', 'df_financiamientos_2019_queretaro', 'df_financiamientos_2019_tulancingo', 'df_financiamientos_2020_puebla', 'df_financiamientos_2020_queretaro', 'df_financiamientos_2020_tulancingo', 'df_financiamientos_2021_puebla', 'df_financiamientos_2021_queretaro', 'df_financiamientos_2021_tulancingo', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2022_puebla', 'df_financiamientos_2022_queretaro', 'df_financiamientos_2022_tulancingo', 'df_financiamientos_2023_pachuca', 'df_financiamientos_2023_puebla', 'df_financiamientos_2023_queretaro', 'df_financiamientos_2023_tulancingo', 'df_grupos_edad_pachuca', 

________
# ***Piramide Poblacional***
# CONSULTAS
### Creación de tablas a usar


In [4]:
datos_grupos_edad_puebla = df_grupos_edad_puebla[['id', 'rango', 'total', 'hombres', 'mujeres']]

In [12]:
# Función para asignar el rango amplio basado en el ID
def asignar_rango_amplio(id):
    rangos = {range(1, 5): '0-19', range(5, 7): '20-34', range(7, 10): '35-49', range(10, 13): '50-64'}
    for rango, etiqueta in rangos.items():
        if id in rango:
            return etiqueta
    return '65+' if id > 12 else 'Desconocido'

# Asumiendo que df_grupos_edad_pachuca ya tiene la columna 'id'
df_grupos_edad_pachuca['rango_amplio'] = df_grupos_edad_pachuca['id'].apply(asignar_rango_amplio)

# Agrupar y sumar en una línea
datos_agrupados = df_grupos_edad_pachuca.groupby('rango_amplio').sum().reset_index()

# Crear columna con la suma de hombres y mujeres
datos_agrupados['total'] = datos_agrupados['Hombres'] + datos_agrupados['Mujeres']

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Función para crear una gráfica de barras
def create_population_bar_chart(df, title, total_col, age_group_col):
    age_groups = df[age_group_col].tolist()
    total_values = df[total_col].tolist()

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=age_groups,
        y=total_values,
        name='Total',
        marker=dict(color=colores),
        textposition='outside',
    ))

    # Configurar el fondo de la gráfica como transparente
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #     'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    # })

    # Personalización adicional del gráfico
    fig.update_layout(
        title=title,
        xaxis=dict(title='Rango de Edad'),
        yaxis=dict(title='Cantidad',gridcolor='#dddcda'),
        legend=dict(x=0.029, y=1.038, font_size=10),
        margin=dict(l=100, r=100, t=100, b=70),
        # paper_bgcolor='white',
        plot_bgcolor='rgba(0,0,0,0)',
    )

    return fig

# Crear y mostrar la gráfica de barras
fig = create_population_bar_chart(datos_agrupados, '', 'total', 'rango_amplio')
fig.show()


In [15]:
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')
    
fig_piramide = create_population_bar_chart(datos_agrupados, '', 'total', 'rango_amplio')

# Guardar la figura como archivo HTML
guardar_grafico_como_html(fig_piramide, 'g_bar_porcentaje_vivienda_ofertadas_antigüedad_puebla')
